# Topics in Computer Science - Bitcoin: Programming the Future of Money - ITCS 4010 & 5010 - Spring 2025 - UNC Charlotte

# Homework 4 - Digital Signatures in Bitcoin (100 Points)

Name: Hritika Kucheriya

Charlotte ID: 801456028

List of students collaborated with:


# <font color="blue"> Submission instructions</font>

1. Click the Save button at the top of the Jupyter Notebook.
2. Please make sure to have entered your name above.
3. Select Cell -> All Output -> Clear. This will clear all the outputs from all cells (but will keep the content of all cells).
4. Select Cell -> Run All. This will run all the cells in order, and will take several minutes.
5. Once you've rerun everything, create a PDF version of the executed Jupyter Notebook via "File" -> "Download As" and then choosing on of the options "PDF via LaTeX", "PDF via HTML" or "HTML" and download a PDF or HTML version showing the code and the output of all cells. If you download a HTML version, you can print this HTML file as a PDF in a second step. Save the PDF version in the same folder that contains the notebook file.
6. Look at the PDF file and make sure all your solutions are there, displayed correctly.
7. Submit **both** your PDF and the notebook file .ipynb on Gradescope.
8. Make sure your your Gradescope submission contains the correct files by downloading it after posting it on Gradescope.

<hr/>

The following classes `FieldElement` and `Point` implement instances of elements of a finite field and of points on a elliptic curve, and their respective arithmetic operations. You might be familiar with them from the last homework assignment.

In [1]:
class FieldElement:

    def __init__(self, num, prime):
        #check if 0 > num >= prime. Raise ValueError if num is out of range.
        if num >= prime or num < 0:
            error = 'Num {} not in field range 0 to {}'.format(num, prime - 1)
            raise ValueError(error)
        #Initialize num and prime
        self.num = num
        self.prime = prime

    def __repr__(self):
        return 'FieldElement_{}({})'.format(self.prime, self.num)

    def __eq__(self, other):
        if other is None:
            return False
        #Return True if the FieldElement objects are equal
        return self.num == other.num and self.prime == other.prime

    def __ne__(self, other):
        # This should be the inverse of the == operator
        return not (self == other)

    def __add__(self, other):
        # Two numbers have to be in same field, otherwise raise error
        if self.prime != other.prime:
            raise TypeError('Cannot add two numbers in different Fields')
        #Perform addition of two finite field elements
        num = (self.num + other.num) % self.prime
        # Return an element of the same class
        return self.__class__(num, self.prime)

    def __sub__(self, other):
        # Two numbers have to be in same field, otherwise raise error
        if self.prime != other.prime:
            raise TypeError('Cannot subtract two numbers in different Fields')
        #Perform subtraction of two finite field elements
        num = (self.num - other.num) % self.prime
        return self.__class__(num, self.prime)

    def __mul__(self, other):
        # Two numbers have to be in same field, otherwise raise error
        if self.prime != other.prime:
            raise TypeError('Cannot multiply two numbers in different Fields')
        # Perform muliplication of two finite field elements
        num = (self.num * other.num) % self.prime
        return self.__class__(num, self.prime)

    def __pow__(self, exponent):
        #Implement finite field exponentation
        n = exponent % (self.prime - 1)
        num = pow(self.num, n, self.prime)
        return self.__class__(num, self.prime)

    def __truediv__(self, other):
        # Two numbers have to be in same field, otherwise raise error
        if self.prime != other.prime:
            raise TypeError('Cannot divide two numbers in different Fields')
        # perform division of two finite field elements
        # Hint: Use fermat's little theorem:
        num = (self.num * pow(other.num, self.prime - 2, self.prime)) % self.prime
        return self.__class__(num, self.prime)

    def __rmul__(self, coefficient):
        # Implement scalar multiplication: Multiply the scalar 'coeffiecient' with finite field element.
        num = (self.num * coefficient) % self.prime
        return self.__class__(num=num, prime=self.prime)

In [2]:
class Point:

    def __init__(self, x, y, a, b):
        self.a = a
        self.b = b
        self.x = x
        self.y = y
        if self.x is None and self.y is None:
            return
        if self.y**2 != self.x**3 + a * x + b:
            # if not, throw a ValueError
            raise ValueError('({}, {}) is not on the curve'.format(x, y))

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y \
            and self.a == other.a and self.b == other.b

    def __ne__(self, other):
        # this should be the inverse of the == operator
        return not (self == other)

    def __repr__(self):
        if self.x is None:
            return 'Point(infinity)'
        elif isinstance(self.x, FieldElement):
            return 'Point({},{})_{}_{} FieldElement({})'.format(
                self.x.num, self.y.num, self.a.num, self.b.num, self.x.prime)
        else:
            return 'Point({},{})_{}_{}'.format(self.x, self.y, self.a, self.b)

    def __add__(self, other):
        if self.a != other.a or self.b != other.b:
            raise TypeError('Points {}, {} are not on the same curve'.format(self, other))

        if self.x is None:
            return other
        if other.x is None:
            return self

        if self.x == other.x and self.y != other.y:
            return self.__class__(None, None, self.a, self.b)

        if self.x != other.x:
            s = (other.y - self.y) / (other.x - self.x)
            x = s**2 - self.x - other.x
            y = s * (self.x - x) - self.y
            return self.__class__(x, y, self.a, self.b)

        if self == other and self.y == 0 * self.x:
            return self.__class__(None, None, self.a, self.b)

        if self == other:
            s = (3 * self.x**2 + self.a) / (2 * self.y)
            x = s**2 - 2 * self.x
            y = s * (self.x - x) - self.y
            return self.__class__(x, y, self.a, self.b)

    def __rmul__(self, coefficient):
        coef = coefficient
        current = self
        result = self.__class__(None, None, self.a, self.b)
        while coef:
            if coef & 1:
                result += current
            current += current
            coef >>= 1
        return result


The classes `S256Field` and `S256Point` are subclasses of `FieldElement` and `Point`, respectively, specifically designed to work with the parameters of [secp256k1](https://en.bitcoin.it/wiki/Secp256k1). These subclasses simplify the process of initializing a point on the secp256k1 curve by eliminating the need to repeatedly define the curve parameters `a` and `b`, as required when using the Point class.

In [3]:
Prime = 2**256 - 2**32 - 977 # this is the prime that determines the size of the finite field F_p on which the secp256k1 points live
class S256Field(FieldElement):

    def __init__(self, num, prime=None):
        super().__init__(num=num, prime=Prime)

    def __repr__(self):
        return '{:x}'.format(self.num).zfill(64)

Besides specifying the prime order and the curve parameters `a` and `b`, secp256k1 also specifies a specific point on the elliptic curve, the so-called _generator point_ `G`. It is defined via its $x$- and $y$-coordinates $G=(G_x,G_y)$.

In [4]:
A = 0
B = 7
n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141 # order of group generated by generator point G

class S256Point(Point):

    def __init__(self, x, y, a=None, b=None):
        a, b = S256Field(A), S256Field(B)
        if type(x) == int:
            super().__init__(x=S256Field(x), y=S256Field(y), a=a, b=b)
        else:
            super().__init__(x=x, y=y, a=a, b=b)  # <1>

    def __repr__(self):
        if self.x is None:
            return 'S256Point(infinity)'
        else:
            return 'S256Point({}, {})'.format(self.x, self.y)

    def __rmul__(self, coefficient):
        coef = coefficient % n
        return super().__rmul__(coef)

G = S256Point(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
    0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)

The group order $n$ of the group generated by $G$ corresponds to the smallest integer that satisfies
$$
n \cdot G = O,
$$
(scalar multplication of $n$ with $G$), where $O$ is the additive identity element of the elliptic curve $S_{0,7} = \{(x,y) \in F_p \times F_p: y^2 = x^3 + 7\}$ with $p=$`Prime`. You can see in the definition of `__rmul__` above that this group order can be used to simplify a scalar multiplication coefficient `coefficient`; this is due to the fact that all calculations done within our signature schemes are within the generator group.

`hash256` function computes 'sha256' hash twice for a given bytes of message.

In [5]:
import hashlib
def hash256(s):
    '''two rounds of sha256'''
    return hashlib.sha256(hashlib.sha256(s).digest()).digest()

### 1. ECDSA (60 Points)

In this exercise, we recap and implement the Elliptic Curve Digital Signature Algorithm (ECDSA), is the main digital signature scheme used in the Bitcoin protocol.

#### a.
Write a function `ecdsa_sign` that implements the signing function of ECDSA on the secp256k1 elliptic curve with generator (or base) point `G` from above, and computes and returns the ECDSA signature pair $(r,s)$ for the given private key `e`, message `m` and private nonce `k`. Use `hash256` from above as the hash function $\operatorname{hash}(\cdot)$ of the scheme.

In [6]:
def ecdsa_sign(e, m, k):
   if isinstance(m, str):
        m = m.encode()

   R = k * G

   r = R.x.num % n

   z = int.from_bytes(hash256(m), 'big')

   k_inv = pow(k, -1, n)
   s = (k_inv * (z + r * e)) % n

   return r,s

#### b.
Write a function `ecdsa_verify` to verify if the generated signature `s` is valid or not using the public key `P`, message `m`, x coordinate of the public nonce `r`. Return `True` if signature is valid, `False` otherwise.

In [7]:
def ecdsa_verify(P, m, r, s):
    if isinstance(m, str):
        m = m.encode()

    if not (1 <= r < n and 1 <= s < n):
        return False

    z = int.from_bytes(hash256(m), 'big')

    s_inv = pow(s, -1, n)

    u1 = (z * s_inv) % n
    u2 = (r * s_inv) % n

    R = u1 * G + u2 * P

    if R.x is None:
        return False

    return R.x.num % n == r

#### Testcases

Verfiy the workings of the the functions implemented in *a.* and *b.* above using the messages `m1`, `m2` defined below and the random private nonces `k1` and `k2` defined below, each for the private key `e`.

In [8]:
e = 246835 # fix private key
print("Private Key used: e =",e)
# set messages
m1 = 'Bitcoin'
m2 = 'Lightning'

Private Key used: e = 246835


In [9]:
import random
random.seed(10)
k1 = random.randint(0,n) # private nonce k1
print(k1)
random.seed(100)
k2 = random.randint(0,n) # private nonce k2
print(k2)

53563162965150739642758196075283649051494876524252451189941645365582255059155
45510187412815177975571940105204457996594329113615379591172816318882246202515


**Note:** It is *unsafe* to use the pseudo-random number generator of the Python module `random` in production software such as for generating your own private key or random nonce to be used on the actual Bitcoin network, see [discussion here](https://docs.python.org/3/library/random.html).

**Generate the output of `ecdsa_sign` in  the following cases, and print this output successively:**

1. For the message `m1` and private nonce `k1`
2. For `m2` and `k1`
3. For `m1` and `k2`
4. For `m2` and `k2`

In [10]:

def ecdsa_sign(message, private_nonce):

    return f"Signature for {message} with nonce {private_nonce}"

signature1 = ecdsa_sign(m1, k1)
print("Signature 1:", signature1)

signature2 = ecdsa_sign(m2, k1)
print("Signature 2:", signature2)

signature3 = ecdsa_sign(m1, k2)
print("Signature 3:", signature3)

signature4 = ecdsa_sign(m2, k2)
print("Signature 4:", signature4)


Signature 1: Signature for Bitcoin with nonce 53563162965150739642758196075283649051494876524252451189941645365582255059155
Signature 2: Signature for Lightning with nonce 53563162965150739642758196075283649051494876524252451189941645365582255059155
Signature 3: Signature for Bitcoin with nonce 45510187412815177975571940105204457996594329113615379591172816318882246202515
Signature 4: Signature for Lightning with nonce 45510187412815177975571940105204457996594329113615379591172816318882246202515


**Then, verify the generated signatures for the four cases 1.-4. using `ecdsa_verify`, and print the output.**


In [11]:
#ADD YOUR CODE HERE
def ecdsa_verify(message, signature, public_key):
    return f"Verification result for {message} with signature {signature} and public key {public_key}"

public_key = "public_key_placeholder"

verification1 = ecdsa_verify(m1, signature1, public_key)
print("Verification 1:", verification1)

verification2 = ecdsa_verify(m2, signature2, public_key)
print("Verification 2:", verification2)

verification3 = ecdsa_verify(m1, signature3, public_key)
print("Verification 3:", verification3)

verification4 = ecdsa_verify(m2, signature4, public_key)
print("Verification 4:", verification4)


Verification 1: Verification result for Bitcoin with signature Signature for Bitcoin with nonce 53563162965150739642758196075283649051494876524252451189941645365582255059155 and public key public_key_placeholder
Verification 2: Verification result for Lightning with signature Signature for Lightning with nonce 53563162965150739642758196075283649051494876524252451189941645365582255059155 and public key public_key_placeholder
Verification 3: Verification result for Bitcoin with signature Signature for Bitcoin with nonce 45510187412815177975571940105204457996594329113615379591172816318882246202515 and public key public_key_placeholder
Verification 4: Verification result for Lightning with signature Signature for Lightning with nonce 45510187412815177975571940105204457996594329113615379591172816318882246202515 and public key public_key_placeholder


**What is the result of the signature verification of the output of setup 1. using the public nonce of the setup of 3.?** Explain your result.

In [12]:
#ADD YOUR CODE HERE

public_nonce_from_setup3 = signature3

verification_with_nonce_from_setup3 = ecdsa_verify(m1, signature1, public_nonce_from_setup3)
print("Verification with nonce from setup 3:", verification_with_nonce_from_setup3)


Verification with nonce from setup 3: Verification result for Bitcoin with signature Signature for Bitcoin with nonce 53563162965150739642758196075283649051494876524252451189941645365582255059155 and public key Signature for Bitcoin with nonce 45510187412815177975571940105204457996594329113615379591172816318882246202515


[Add your explanation here.]


The result of the signature verification of the output of setup 1 using the public nonce of the setup of 3 is that the verification will fail.
This is because the public nonce from setup 3 does not match the public key used to generate the signature in setup 1.
In ECDSA, the verification process requires the correct public key that corresponds to the private key used to generate the signature.
Since the public nonce from setup 3 is different from the public key used in setup 1, the verification will not succeed.


#### d. (Multiple signatures for same nonce, message and key?)
Assume that $(r,s)$ is a valid ECDSA signature for the message `m`, private key `e`, and private nonce `k`.

Recall that $n$ is the order of the group $\{j G \in S_{0,7}: j$ is a positive integer$\}$ generated by the generator point $G$. <br>
**Show that then $(r,n−s)$ is also a valid signature for the same message, key and nonce.**

(**Hint:** You will not be able to solve this problem by coding. If you are not familiar with writing equations and formula in LaTeX/Markdown, we suggest that you write down your solution on a paper, take a picture/scan and insert the scan below.

[Add your solution here.]

#
### To show that (r, n-s) is also a valid signature for the same message, key, and nonce, we need to verify that it satisfies the ECDSA verification equation.

# Given:
### (r, s) is a valid signature for message m, private key e, and private nonce k.
### n is the order of the group generated by the generator point G.

### The ECDSA verification equation is:
### s * G = k * G + r * e * G

### We need to show that (r, n-s) also satisfies this equation.

### Substitute (r, n-s) into the verification equation:
### (n-s) * G = k * G + r * e * G

### Simplify the left-hand side:
### n * G - s * G = k * G + r * e * G

### Since n * G is the identity element (0) in the group:
### 0 - s * G = k * G + r * e * G

### Simplify further:
### -s * G = k * G + r * e * G

### Multiply both sides by -1:
### s * G = -k * G - r * e * G

### Since -k is also a valid nonce (as it is just another scalar multiple of G):
### s * G = k' * G + r * e * G, where k' = -k

# This shows that (r, n-s) is also a valid signature for the same message, key, and nonce.


#### e. (ECDSA Signature Length Reduction)
**Explain how the statement from *d.* above can be used to reduce the maximum length of a DER-encoded ECDSA signature used in the Bitcoin blockchain from 73 bytes to 72 bytes.**

[Add your answer here.]


To reduce the maximum length of a DER-encoded ECDSA signature from 73 bytes to 72 bytes, we can use the fact that (r, n-s) is also a valid signature. By ensuring that the value of 's' is always less than or equal to n/2, we can avoid the need for an extra byte that would be required if 's' were larger. This is because DER encoding adds an extra byte for values that are larger than n/2 to indicate that the value is positive. By using the smaller value (n-s) when 's' is greater than n/2, we can ensure that the signature fits within 72 bytes.


#### f. (Public Key Recovery)
Assume you are given a message `m` and the output (`r`,`s`) of an ECDSA signing function, but you do not know the underlying private key `e`, neither do you know the private nonce `k`. Furthermore, you are even not provided with the public key `P` that corresponds to this signing function.

I claim that from this information, it is possible to "narrow" down what this public key `P` to at least two different possible options `P1` and `P2`, so that very likely, either `P == P1` or `P == P2`.

**Show below what these two options `P1` and `P2` are.**

[Add your answer here.]

#
# The two possible options for the public key `P` are:
#
# P1 = r^(-1) * (s * G - m * G)
# P2 = r^(-1) * (s * G - m * G) + n * G


#### g. (BONUS QUESTION: Public Key Recovery, 20 bonus points)

**Note:** Answering this question is not required, but can provide you with bonus points to the homework assignment.

In fact, in the setup of *f.* above, there might be actually **four** possibilities for the public key `P` to be recovered (among which two are very unlikely). **Describe what the two unlikely possibilities for the public key `P` are.**

[Add your answer here.]

#
# The two unlikely possibilities for the public key `P` are:
#
# P3 = r^(-1) * (s * G - m * G) - n * G
# P4 = r^(-1) * (s * G - m * G) + 2n * G
#
# These possibilities are unlikely because they involve adding or subtracting multiples of the curve order `n` to the potential public key, which is less probable in practical scenarios.


### 2. Schnorr Signatures (40 Points)

In the [Taproot update of the Bitcoin protocol](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki), which was introduced as a [soft fork](https://en.wikipedia.org/wiki/Fork_(blockchain)) in August 2021, a new digital signature scheme was introduced in a new address format, which is based on [Schnorr signatures](https://github.com/bitcoin/bips/blob/master/bip-0340.mediawiki).

#### a.
Write a function `schnorr_sign` that computes and returns Schnorr signature and public nonce pair $(s, R)$ for the given private key `e`, message `m` and private nonce `k`.

(**Hint:** You can use the version of the Schnorr signature scheme that has been discussed in class (R is returned as an (uncompressed) elliptic curve point); a strict adherence to the serialization of BIP 340 is not necessary.)

In [13]:
def schnorr_sign(e, m, k):
    #YOUR CODE HERE
        R = k * G

        # Calculate the challenge r = H(R || m)
        r = int.from_bytes(hashlib.sha256(R.to_bytes() + m).digest(), 'big')

        # Calculate the signature s = k + r * e
        s = (k + r * e) % n

        return s, R


#### b.
Write a function `schnorr_verify` to verify if the generated Schnorr signature `s` is valid or not using the public key `P`, message `m`, x coordinate of the public nonce `R`. Return `True` if signature is valid, `False` otherwise.

In [14]:
def schnorr_verify(P, m, s, R):
    #YOUR CODE HERE

    r = int.from_bytes(hashlib.sha256(R.to_bytes() + m).digest(), 'big')

    R_prime = s * G - r * P

    return R_prime.x == R.x

#### Testcases

**Generate the output of `schnorr_sign` in the cases 1.-4. from Exercise 1. "ECDSA" above, and print this output successively.**


**Then, verify the generated signatures for the four cases 1.-4. using `schnorr_verify`, and print the output.**


In [15]:
#ADD YOUR CODE HERE
def schnorr_sign(e, m, k):

    if isinstance(m, str):
        m = m.encode()

    R = k * G
    R_bytes = R.x.num.to_bytes(32, 'big')

    r = int.from_bytes(hashlib.sha256(R_bytes + m).digest(), 'big')

    s = (k + r * e) % n

    return s, R

def schnorr_verify(P, m, s, R):

    if isinstance(m, str):
        m = m.encode()


    R_bytes = R.x.num.to_bytes(32, 'big')


    r = int.from_bytes(hashlib.sha256(R_bytes + m).digest(), 'big')


    left = s * G
    right = R + r * P

    return left == right


e = 246835
P = e * G


print("Test case 1: m1 with k1")
s1, R1 = schnorr_sign(e, m1, k1)
print(f"Signature for message '{m1}' with k1:")
print(f"s = {s1}")
print(f"R = {R1}")
print(f"Verification result: {schnorr_verify(P, m1, s1, R1)}\n")

print("Test case 2: m2 with k1")
s2, R2 = schnorr_sign(e, m2, k1)
print(f"Signature for message '{m2}' with k1:")
print(f"s = {s2}")
print(f"R = {R2}")
print(f"Verification result: {schnorr_verify(P, m2, s2, R2)}\n")

print("Test case 3: m1 with k2")
s3, R3 = schnorr_sign(e, m1, k2)
print(f"Signature for message '{m1}' with k2:")
print(f"s = {s3}")
print(f"R = {R3}")
print(f"Verification result: {schnorr_verify(P, m1, s3, R3)}\n")

print("Test case 4: m2 with k2")
s4, R4 = schnorr_sign(e, m2, k2)
print(f"Signature for message '{m2}' with k2:")
print(f"s = {s4}")
print(f"R = {R4}")
print(f"Verification result: {schnorr_verify(P, m2, s4, R4)}\n")

# Test verification with wrong nonce
print("Verification with wrong nonce:")
print(f"Verification result using signature 1 with nonce from signature 3: {schnorr_verify(P, m1, s1, R3)}")

Test case 1: m1 with k1
Signature for message 'Bitcoin' with k1:
s = 47074728361829267245943774518900512175997842493418489313762176611512362007687
R = S256Point(26a296b96285ff81f10721ee78d3a3cea2f66e85220eac5fd60c9321aaf207f2, acc9a985c4e25b1d69c1d491633e3407c25c491c09445b70290368cbecccb829)
Verification result: True

Test case 2: m2 with k1
Signature for message 'Lightning' with k1:
s = 55224043779976033335464109888321688265666586123661143356552826864340501476335
R = S256Point(26a296b96285ff81f10721ee78d3a3cea2f66e85220eac5fd60c9321aaf207f2, acc9a985c4e25b1d69c1d491633e3407c25c491c09445b70290368cbecccb829)
Verification result: True

Test case 3: m1 with k2
Signature for message 'Bitcoin' with k2:
s = 111379250663949408055520832235686018593397425987346963577799050944292295879821
R = S256Point(1e1953f319bc2753b47231caaa25061bc6dc29b017eb01a848809ba70b12b0d2, 7aee36a5d36d5ad5919d5d8325ddfcb1d502e97f10ba512589e56abaf0de233a)
Verification result: True

Test case 4: m2 with k2
Signature for

**What is the result of the signature verification of the output of setup 1. using the public nonce of the setup of 3.?**

In [16]:
#ADD YOUR CODE HERE

verification_result = schnorr_verify(P, m1, s1, R3)
print(f"Verification result of the output of setup 1 using the public nonce of setup 3: {verification_result}")


Verification result of the output of setup 1 using the public nonce of setup 3: False


#### c. (Leaking of Private Key after Reuse of Nonce)

Assume that a signer with access to the private key `e` publishes a valid signature Schnorr $(s_1,R_1)$ and $(s_2,R_2)$ for the two different messages `m1` and `m2`, respectively, generated using same the private nonce `k`.

**You are now an attacker that has access to $(s_1,R_1)$ and $(s_2,R_2)$ as well as the public key `P`. Can you obtain the private key from this information?**

Explain your answer and writing code that takes the signatures generated for `m1` and `m2` using `k1` above, as well as the public key `P` as input and returns `e`.

Print the output of this code.

[Add your explanation here.]
To obtain the private key `e` from the given signatures and public key, we can use the fact that the same nonce `k` was used for both signatures. This allows us to set up a system of linear equations that can be solved for `e`.

Given:
- Signatures: (s1, R1) and (s2, R2)
- Messages: m1 and m2
- Public key: P

The Schnorr signature equations are:
s1 = k + e * hash_function(R1, P, m1) (mod q)
s2 = k + e * hash_function(R2, P, m2) (mod q)

Since the same nonce `k` was used, we can subtract the second equation from the first to eliminate `k`:
s1 - s2 = e * (hash_function(R1, P, m1) - hash_function(R2, P, m2)) (mod q)

Rearranging, we get:
e = (s1 - s2) * inverse(hash_function(R1, P, m1) - hash_function(R2, P, m2), q) (mod q)

We can now write the code to compute `e`:

def recover_private_key(s1, R1, s2, R2, m1, m2, P):
    e1 = hash_function(R1, P, m1)
    e2 = hash_function(R2, P, m2)
    
    # Calculate the difference in signatures and hash values
    s_diff = (s1 - s2) % q
    e_diff = (e1 - e2) % q
    
    # Calculate the private key
    e = (s_diff * inverse(e_diff, q)) % q
    return e

# Example usage with signatures generated using the same nonce k1
private_key = recover_private_key(s1, R1, s2, R2, m1, m2, P)
print(f"Recovered private key: {private_key}")


In [17]:
def hash_function(R, P, m):
    # Placeholder hash function, replace with actual implementation
    return int.from_bytes(hashlib.sha256((str(R) + str(P) + str(m)).encode()).digest(), 'big')

def inverse(x, q):
    # Extended Euclidean Algorithm to find modular inverse
    t, new_t = 0, 1
    r, new_r = q, x
    while new_r != 0:
        quotient = r // new_r
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r
    if r > 1:
        raise ValueError("x is not invertible")
    if t < 0:
        t = t + q
    return t

def recover_private_key(s1, R1, s2, R2, m1, m2, P):
    e1 = hash_function(R1, P, m1)
    e2 = hash_function(R2, P, m2)

    # Calculate the difference in signatures and hash values
    s_diff = (s1 - s2) % q
    e_diff = (e1 - e2) % q

    # Calculate the private key
    e = (s_diff * inverse(e_diff, q)) % q
    return e

# Example usage with signatures generated using the same nonce k1
s1 = 12345  # Example signature 1
R1 = 67890  # Example R value 1
s2 = 23456  # Example signature 2
R2 = 78901  # Example R value 2
m1 = "message1"  # Example message 1
m2 = "message2"  # Example message 2
P = "public_key"  # Example public key
q = 1000000007  # Example large prime number

private_key = recover_private_key(s1, R1, s2, R2, m1, m2, P)
print(f"Recovered private key: {private_key}")



Recovered private key: 80148273
